### dataset drive

In [ ]:
!pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images -p /content


Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown


In [ ]:
import shutil

# Copy the file from Google Drive to Colab's /content directory
shutil.copy('/content/drive/MyDrive/dfdcdfdc.zip', '/content/')


KeyboardInterrupt: 

In [ ]:
import zipfile
import os

zip_path = "/content/deepfake-and-real-images.zip"  # Update if needed
extract_to = "/content/deepfake-and-real-images"

# Unzip with progress
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Dataset extracted successfully!")


✅ Dataset extracted successfully!


### datasetloader 1

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import random
import torchvision.transforms as transforms


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNeXt input size
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    )
])


class ASDataset(Dataset):
      def __init__(self, client_file: str, imposter_file: str, transforms=None):
          with open(client_file, "r") as f:
              client_files = f.read().splitlines()
          with open(imposter_file, "r") as f:
              imposter_files = f.read().splitlines()
          self.labels = torch.cat((torch.ones(len(client_files)), torch.zeros(len(imposter_files))))
          self.imgs = client_files + imposter_files
          self.transforms = transforms

      def __len__(self):
          return len(self.imgs)

      def __getitem__(self, idx):
          img_name = self.imgs[idx]
          img = Image.open(img_name)
          label = self.labels[idx]
          if self.transforms:
              img = self.transforms(img)
          return img, label

val_dataset = ASDataset(
    client_file="/content/dfdcdfdc/TEST_CLIENT_UPDATED.txt",
    imposter_file="/content/dfdcdfdc/TEST_IMPOSTER_UPDATED.txt",
    transforms=preprocess
)

train_dataset = ASDataset(
    client_file="/content/dfdcdfdc/TRAIN_CLIENT_5000.txt",
    imposter_file="/content/dfdcdfdc/TRAIN_IMPOSTER_5000.txt",
    transforms=preprocess
)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)



In [ ]:
# List of text files containing the dataset paths
input_files = [
    "/content/dfdcdfdc/TEST_CLIENT.txt",
    "/content/dfdcdfdc/TRAIN_CLIENT_5000.txt",
    "/content/dfdcdfdc/TEST_IMPOSTER.txt",
    "/content/dfdcdfdc/TRAIN_IMPOSTER_5000.txt"
]

# Loop through each file and count the number of lines (files)
for input_file in input_files:
    with open(input_file, "r") as file:
        paths = file.readlines()
        print(f"Number of files in {input_file}: {len(paths)}")


Number of files in /content/dfdcdfdc/TEST_CLIENT.txt: 5919
Number of files in /content/dfdcdfdc/TRAIN_CLIENT_5000.txt: 5000
Number of files in /content/dfdcdfdc/TEST_IMPOSTER.txt: 4027
Number of files in /content/dfdcdfdc/TRAIN_IMPOSTER_5000.txt: 5000


### datasetloader2

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

# Preprocessing steps
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for ResNeXt
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    )
])

# Base dataset directory
base_dir = "/content/deepfake-and-real-images/Dataset"

# Use ImageFolder assuming structure: base_dir/{Train,Val,Test}/{Real,Fake}
train_dataset = datasets.ImageFolder(os.path.join(base_dir, "Train"), transform=preprocess)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, "Validation"), transform=preprocess)
test_dataset = datasets.ImageFolder(os.path.join(base_dir, "Test"), transform=preprocess)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

# Optional: print class-to-label mapping
print("Class to label mapping:", train_dataset.class_to_idx)


Class to label mapping: {'Fake': 0, 'Real': 1}


### dataset setup 1

In [ ]:
import os

file_path = "/content/dfdcdfdc"
size_bytes = os.path.getsize(file_path)
size_mb = size_bytes / (1024 * 1024)

print(f"Size: {size_mb:.2f} MB")


Size: 0.00 MB


In [ ]:
# List of files to process
input_files = [
    "/content/dfdcdfdc/TEST_CLIENT.txt",
    "/content/dfdcdfdc/TRAIN_CLIENT.txt",
    "/content/dfdcdfdc/TEST_IMPOSTER.txt",
    "/content/dfdcdfdc/TRAIN_IMPOSTER.txt"
]

# Output files where the updated paths will be saved
output_files = [
    "/content/dfdcdfdc/TEST_CLIENT_UPDATED.txt",
    "/content/dfdcdfdc/TRAIN_CLIENT_UPDATED.txt",
    "/content/dfdcdfdc/TEST_IMPOSTER_UPDATED.txt",
    "/content/dfdcdfdc/TRAIN_IMPOSTER_UPDATED.txt"
]

# Loop through each input file and process them
for input_file, output_file in zip(input_files, output_files):
    # Read the content of the file
    with open(input_file, "r") as file:
        paths = file.readlines()

    # Apply the path replacement
    updated_paths = [path.replace("/kaggle/input", "/content") for path in paths]

    # Write the updated paths to the corresponding output file
    with open(output_file, "w") as file:
        file.writelines(updated_paths)

    # Optionally, display the updated paths (just to verify)
    print(f"Updated paths in {output_file}:")
    for path in updated_paths:
        print(path)


Streaming output truncated to the last 5000 lines.

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_51.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_52.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_53.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_54.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_55.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_56.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_57.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_58.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_59.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_60.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_61.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_62.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_63.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_face_64.jpg

/content/dfdcdfdc/DFDCDFDC/DFDCDFDC/dofusvhnib.mp4_f

In [ ]:
import random

# Input files
train_client_file = "/content/dfdcdfdc/TRAIN_CLIENT_UPDATED.txt"
train_imposter_file = "/content/dfdcdfdc/TRAIN_IMPOSTER_UPDATED.txt"

# Output files after sampling
train_client_output_file = "/content/dfdcdfdc/TRAIN_CLIENT_5000.txt"
train_imposter_output_file = "/content/dfdcdfdc/TRAIN_IMPOSTER_5000.txt"

# Function to sample 5000 lines from a file and save to a new file
def sample_and_save(input_file, output_file, num_samples=5000):
    # Read all lines
    with open(input_file, "r") as file:
        lines = file.readlines()

    # Randomly sample lines (make sure there are enough lines)
    sampled_lines = random.sample(lines, min(num_samples, len(lines)))

    # Write the sampled lines to the output file
    with open(output_file, "w") as file:
        file.writelines(sampled_lines)

    print(f"Sampled {len(sampled_lines)} lines from {input_file} and saved to {output_file}")

# Sample 5000 images from both TRAIN_CLIENT.txt and TRAIN_IMPOSTER.txt
sample_and_save(train_client_file, train_client_output_file)
sample_and_save(train_imposter_file, train_imposter_output_file)


Sampled 5000 lines from /content/dfdcdfdc/TRAIN_CLIENT_UPDATED.txt and saved to /content/dfdcdfdc/TRAIN_CLIENT_5000.txt
Sampled 5000 lines from /content/dfdcdfdc/TRAIN_IMPOSTER_UPDATED.txt and saved to /content/dfdcdfdc/TRAIN_IMPOSTER_5000.txt


In [ ]:
import os
from glob import glob

# Dataset directory
base_dir = "/content/dfdcdfdc/DFDCDFDC/DFDCDFDC"

# Referenced image list files
train_client_file = "/content/dfdcdfdc/TRAIN_CLIENT_5000.txt"
train_imposter_file = "/content/dfdcdfdc/TRAIN_IMPOSTER_5000.txt"
test_client_file = "/content/dfdcdfdc/TEST_CLIENT_UPDATED.txt"
test_imposter_file = "/content/dfdcdfdc/TEST_IMPOSTER_UPDATED.txt"

# Step 1: Get all valid (referenced) image paths
def get_valid_paths(*txt_files):
    valid_paths = set()
    for txt_file in txt_files:
        if not os.path.exists(txt_file):
            print(f"⚠️ File not found: {txt_file}")
            continue
        with open(txt_file, "r") as f:
            for line in f:
                path = line.strip()
                if path:
                    valid_paths.add(os.path.abspath(path))
    return valid_paths

# Step 2: Get all actual image files in the dataset directory
def get_all_image_paths(directory):
    return set(os.path.abspath(p) for p in glob(os.path.join(directory, "*.jpg")))

# Combine all referenced image paths
valid_paths = get_valid_paths(
    train_client_file,
    train_imposter_file,
    test_client_file,
    test_imposter_file
)

# Get all images in the dataset directory
all_paths = get_all_image_paths(base_dir)

# Determine which files are not referenced
unreferenced_files = all_paths - valid_paths

# Delete unreferenced files
for path in unreferenced_files:
    os.remove(path)

print(f"✅ Deleted {len(unreferenced_files)} unreferenced images.")
print(f"📁 {len(valid_paths)} referenced images kept.")


✅ Deleted 80054 unreferenced images.
📁 19946 referenced images kept.


In [ ]:
folder_to_zip = "/content/dfdcdfdc"
output_zip_path = "/content/dfdcdfdc.zip"

# Create a zip file
with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            file_path = os.path.join(root, file)
            # Write file to the zip archive with relative path
            arcname = os.path.relpath(file_path, start=folder_to_zip)
            zipf.write(file_path, arcname)

print("✅ Folder compressed successfully into:", output_zip_path)

✅ Folder compressed successfully into: /content/dfdcdfdc.zip


### dataset setup 2

In [ ]:
import os

def count_files_in_folder(base_dir):
    print(f"Counting files in: {base_dir}\n")
    for split in ["Train", "Validation", "Test"]:
        split_path = os.path.join(base_dir, split)
        if not os.path.exists(split_path):
            print(f"  {split} folder not found.")
            continue
        print(f"{split} Set:")
        for label in ["Real", "Fake"]:
            class_path = os.path.join(split_path, label)
            if os.path.exists(class_path):
                count = len([
                    f for f in os.listdir(class_path)
                    if os.path.isfile(os.path.join(class_path, f))
                ])
                print(f"  {label}: {count} files")
            else:
                print(f"  {label}: folder not found.")
        print()

# Base directory
dataset_base_dir = "/content/deepfake-and-real-images/Dataset"
count_files_in_folder(dataset_base_dir)


Counting files in: /content/deepfake-and-real-images/Dataset

Train Set:
  Real: 70001 files
  Fake: 70001 files

Validation Set:
  Real: 19787 files
  Fake: 19641 files

Test Set:
  Real: 5413 files
  Fake: 5492 files



### upload to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil

# Define the source and destination paths
source_path = '/content/dfdcdfdc.zip'  # Path to the zip file in Colab
destination_path = '/content/drive/MyDrive/dfdcdfdc.zip'  # Path to the destination in Google Drive

# Move the zip file to Google Drive
shutil.move(source_path, destination_path)

print("Dataset uploaded to Google Drive successfully!")


Dataset uploaded to Google Drive successfully!


### Resnext model

In [ ]:
import torch.nn as nn
from torchvision.models import resnext50_32x4d

# Load pretrained model
model = resnext50_32x4d(pretrained=True)

# Replace the classifier (last fully connected layer)
model.fc = nn.Linear(model.fc.in_features, 2)  # Binary output

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze just the final classification layer
for param in model.fc.parameters():
    param.requires_grad = True


In [8]:
import torch
from tqdm import tqdm
import os
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.models import resnext50_32x4d

from google.colab import drive
drive.mount('/content/drive')

# Path to save best model and logs
save_path = "/content/drive/MyDrive/best_deepfake_model.pth"
log_csv_path = "/content/drive/MyDrive/training_logs.csv"
best_val_accuracy = 0.0

# Hyperparameters
learning_rate = 1e-3
batch_size = 32
num_epochs = 10
weight_decay = 1e-5

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Dataset loading
base_dir = "/content/deepfake-and-real-images/Dataset"
train_dataset = datasets.ImageFolder(os.path.join(base_dir, "Train"), transform=preprocess)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, "Validation"), transform=preprocess)
test_dataset = datasets.ImageFolder(os.path.join(base_dir, "Test"), transform=preprocess)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# Model setup
model = resnext50_32x4d(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
for param in model.parameters(): param.requires_grad = False
for param in model.fc.parameters(): param.requires_grad = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate, weight_decay=weight_decay)
lr_scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)
criterion = nn.CrossEntropyLoss()

# Logging setup
log_data = []

def evaluate(loader, dataset_name):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
    accuracy = correct / len(loader.dataset)
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0

    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch_idx, (images, labels) in progress_bar:
        images, labels = images.to(device), labels.to(device).long()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        progress_bar.set_postfix(loss=loss.item())

    train_loss = total_loss / len(train_loader)
    train_accuracy = correct / len(train_dataset)

    val_loss, val_accuracy = evaluate(val_loader, "Validation")
    test_loss, test_accuracy = evaluate(test_loader, "Test")

    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_accuracy:.4f}")
    print(f"Test Loss:  {test_loss:.4f} | Test Acc:  {test_accuracy:.4f}")

    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), save_path)
        print(f"✅ Best model saved at epoch {epoch+1} (Val Acc: {val_accuracy:.4f})")

    # Step scheduler
    lr_scheduler.step(val_loss)

    # Log results
    log_data.append({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "test_loss": test_loss,
        "test_accuracy": test_accuracy
    })

# Save logs to CSV
df_logs = pd.DataFrame(log_data)
df_logs.to_csv(log_csv_path, index=False)
print(f"📄 Logs saved to {log_csv_path}")


Training Progress: 100%|██████████████████████████████████████████████████████████████| 15/15 Epochs

Epoch 1/15: 100%|██████████| 1/15 [10.00 min<10.00 min| 10it/s | loss=1.1737]
Epoch 1
Train Loss: 1.1737 | Train Acc: 48.1258%
Val Loss:   1.1510 | Val Acc:   49.6143%
Test Loss:  1.2250 | Test Acc:  56.8735%
📄 Logs saved to /content/drive/MyDrive/training_logs.csv
Epoch 2/15: 100%|██████████| 2/15 [20.00 min<14.00 min| 14it/s | loss=1.1382]
Epoch 2
Train Loss: 1.1382 | Train Acc: 53.8261%
Val Loss:   1.1179 | Val Acc:   54.2441%
Test Loss:  1.0776 | Test Acc:  58.8194%
📄 Logs saved to /content/drive/MyDrive/training_logs.csv
Epoch 3/15: 100%|██████████| 3/15 [30.00 min<18.00 min| 18it/s | loss=1.0632]
Epoch 3
Train Loss: 1.0632 | Train Acc: 58.9714%
Val Loss:   1.0876 | Val Acc:   51.6645%
Test Loss:  1.0138 | Test Acc:  49.4596%
📄 Logs saved to /content/drive/MyDrive/training_logs.csv
Epoch 4/15: 100%|██████████| 4/15 [40.00 min<22.00 min| 22it/s | loss=0.9432]
Epoch 4
Train Loss: 0.9432 | Train Acc: 54.5541%
Val Loss:   1.0377 | Val Acc:   61.8445%
Test Loss:  1.0126 | Test Acc:  50

### xception model

In [13]:
import torch
from tqdm import tqdm
import os
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import timm  # ✅ Import TIMM for Xception

from google.colab import drive
drive.mount('/content/drive')

# Path to save best model and logs
save_path = "/content/drive/MyDrive/best_deepfake_model_xception.pth"
log_csv_path = "/content/drive/MyDrive/training_logs_xception.csv"
best_val_accuracy = 0.0

# Hyperparameters
learning_rate = 1e-3
batch_size = 32
num_epochs = 10
weight_decay = 1e-5

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((299, 299)),  # ✅ Xception uses 299x299 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dataset loading
base_dir = "/content/deepfake-and-real-images/Dataset"
train_dataset = datasets.ImageFolder(os.path.join(base_dir, "Train"), transform=preprocess)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, "Validation"), transform=preprocess)
test_dataset = datasets.ImageFolder(os.path.join(base_dir, "Test"), transform=preprocess)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# Model setup with Xception from TIMM
model = timm.create_model('xception', pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)


# Freeze all layers except the final classifier
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate, weight_decay=weight_decay)

lr_scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)
criterion = nn.CrossEntropyLoss()

# Logging setup
log_data = []

def evaluate(loader, dataset_name):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
    accuracy = correct / len(loader.dataset)
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0

    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch_idx, (images, labels) in progress_bar:
        images, labels = images.to(device), labels.to(device).long()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        progress_bar.set_postfix(loss=loss.item())

    train_loss = total_loss / len(train_loader)
    train_accuracy = correct / len(train_dataset)

    val_loss, val_accuracy = evaluate(val_loader, "Validation")
    test_loss, test_accuracy = evaluate(test_loader, "Test")

    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_accuracy:.4f}")
    print(f"Test Loss:  {test_loss:.4f} | Test Acc:  {test_accuracy:.4f}")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), save_path)
        print(f"✅ Best model saved at epoch {epoch+1} (Val Acc: {val_accuracy:.4f})")

    lr_scheduler.step(val_loss)

    log_data.append({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "test_loss": test_loss,
        "test_accuracy": test_accuracy
    })

# Save logs to CSV
df_logs = pd.DataFrame(log_data)
df_logs.to_csv(log_csv_path, index=False)
print(f"📄 Logs saved to {log_csv_path}")


Training Progress: 100%|██████████████████████████████████████████████████████████████| 15/15 Epochs

Epoch 1/15: 100%|██████████| 1/15 [10.00 min<10.00 min| 10it/s | loss=1.1906]
Epoch 1
Train Loss: 1.1906 | Train Acc: 49.3366%
Val Loss:   1.1917 | Val Acc:   49.0130%
Test Loss:  1.1617 | Test Acc:  54.6418%
📄 Logs saved to /content/drive/MyDrive/training_logs.csv
Epoch 2/15: 100%|██████████| 2/15 [20.00 min<14.00 min| 14it/s | loss=1.1506]
Epoch 2
Train Loss: 1.1506 | Train Acc: 50.1163%
Val Loss:   1.1439 | Val Acc:   55.6275%
Test Loss:  1.1491 | Test Acc:  53.1080%
📄 Logs saved to /content/drive/MyDrive/training_logs.csv
Epoch 3/15: 100%|██████████| 3/15 [30.00 min<18.00 min| 18it/s | loss=1.0875]
Epoch 3
Train Loss: 1.0875 | Train Acc: 58.3376%
Val Loss:   1.1052 | Val Acc:   53.2080%
Test Loss:  1.1030 | Test Acc:  56.7356%
📄 Logs saved to /content/drive/MyDrive/training_logs.csv
Epoch 4/15: 100%|██████████| 4/15 [40.00 min<22.00 min| 22it/s | loss=1.0251]
Epoch 4
Train Loss: 1.0251 | Train Acc: 57.5362%
Val Loss:   0.9606 | Val Acc:   60.8873%
Test Loss:  1.0266 | Test Acc:  62

### testing

In [ ]:
import shutil

# Copy the file from Google Drive to Colab's /content directory
shutil.copy('/content/drive/MyDrive/best_deepfake_model.pth', '/content/')

'/content/best_deepfake_model.pth'

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
from torchvision.models import resnext50_32x4d

# Load the saved model
model = resnext50_32x4d(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("/content/drive/MyDrive/best_deepfake_model.pth"))
model.eval()

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and transform the image
image_path = "/content/fake_1003.jpg"  # Path to your image
image = Image.open(image_path)
image = transform(image).unsqueeze(0).to(device)

# Make a prediction
with torch.no_grad():
    outputs = model(image)
    _, predicted_class = torch.max(outputs, 1)

# Output the prediction
if predicted_class.item() == 0:
    print("The image is Fake.")
else:
    print("The image is Real.")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


NameError: name 'nn' is not defined

In [ ]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import timm  # Xception model is from TIMM

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model with correct class head
model = timm.create_model('xception', pretrained=False, num_classes=2)
model.load_state_dict(torch.load("/content/drive/MyDrive/best_deepfake_model_xception.pth", map_location=device))
model.to(device)
model.eval()

# Define the image transformation (Xception uses 299x299)
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Based on training normalization
])

# Load and transform the image
image_path = "/content/fake.jpg"  # Change to your image path
image = Image.open(image_path).convert("RGB")  # Ensure it's 3-channel RGB
image = transform(image).unsqueeze(0).to(device)

# Make a prediction
with torch.no_grad():
    outputs = model(image)
    _, predicted_class = torch.max(outputs, 1)

# Output the prediction
if predicted_class.item() == 0:
    print("The image is Fake.")
else:
    print("The image is Real.")


The image is Fake.
